In [5]:
# Para el conjunto de datos Wine del ejercicio anterior, realiza un modelo de ensamble de árboles de decisión donde se agrupen 500 modelos predictores
# entrenados con 75 observaciones de la muestra de entrenamiento cada uno. Para poder validar el modelo, divide el conjunto de datos en 128 
# observaciones para la muestra de entrenamiento y 50 para la muestra de validación.
# a) Con reemplazamiento (bagging)
# b) Sin reemplazamiento (pasting)
# c) Realizando Random Forest (fijando el número máximo de nodos hoja en 4)
# ¿Con qué método se obtiene la mejor predicción?

In [8]:
# Cada modelo predictor del ensamble debe emplear 75 de las 128 observadiones de entrenamiento

# Se comienza cargando el dataset
import pandas as pd
from sklearn.datasets import load_wine # Para importar el conjunto de datos

wine = load_wine(as_frame=True) # Se importa el conjunto de datos como un dataframe (as_frame=True)

# Variables predictoras
print(wine.data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
dtypes: fl

In [36]:
# Se divide el dataset en un conjunto de entrenamiento y un conjunto de test. El conjunto de entrenamiento está formado por 128 datos de los 178
# totales. El tanto por uno para el conjunto de test es (178-128)/178.
import numpy as np
# Función para crear el conjunto de entrenamiento y el conjunto de validación
# 'dataset' es el dataframe y 'porcentaje_validacion' es la proporción en tanto por un 1 para el conjunto de validación.
def particiones(target, dataset, porcentaje_validacion):
    
    # 'int' se queda con la parte entera. Con len(dataset) se calcula la longitud de cada columna
    tamaño_validacion = int(round(len(dataset)*porcentaje_validacion,0))

    # Crea un array con los índices de 'dataset' dispuestos de forma aleatoria
    mezclar_indices = np.random.permutation(len(dataset))

    # Se queda con tantos índices del array anterior como datos debe haber en el conjunto de validación
    indices_validacion = mezclar_indices[:tamaño_validacion]

    # Los índices restantes son para los datos del dataset de entrenamiento
    indices_entrenamiento = mezclar_indices[tamaño_validacion:]

    # Se devuelven los dataset de entrenamiento y de test. 'iloc' saca filas
    return dataset.iloc[indices_entrenamiento], dataset.iloc[indices_validacion], target.iloc[indices_entrenamiento], target.iloc[indices_validacion]

# Se llama a la función para crear el conjunto de entrenamiento (train) y el conjunto de validación (test). 
X_train, X_test, y_train, y_test = particiones(wine.target, wine.data, (178-128)/178)

# Se comprueba que la partición es correcta calculando la cantidad de datos en cada conjunto
print('Cantidad teórica de datos en el conjunto de entrenamiento:', round(len(wine.data)*128/178,0))
print('Cantidad de datos en el conjunto de entrenamiento creado:', len(train))
print('Cantidad teórica de datos en el conjunto de validación:', round(len(wine.data)*(178-128)/178,0))
print('Cantidad de datos en el dataset de test creado:', len(test))

Cantidad teórica de datos en el conjunto de entrenamiento: 128.0
Cantidad de datos en el conjunto de entrenamiento creado: 128
Cantidad teórica de datos en el conjunto de validación: 50.0
Cantidad de datos en el dataset de test creado: 50


In [39]:
# Apartado a)
# Se utiliza el método de ensamble tipo bagging con el modelo árbol de decisión
from sklearn.ensemble import BaggingClassifier # Para realizar ensamble tipo bagging
from sklearn.tree import DecisionTreeClassifier # Para crea árbol de decisión
from sklearn.metrics import accuracy_score # Para medir bondad del ajuste

# Se crea objeto BaggingClassifier
# Con bootstrap = True se establece que el método de ensamble es tipo bagging
bag_clf = BaggingClassifier(DecisionTreeClassifier(random_state=3), n_estimators=500, max_samples=75, bootstrap=True, random_state=3)

# Se ajusta el modelo
bag_clf.fit(X_train, y_train)

# Estimaciones del modelo sobre el conjunto de validación
y_pred = bag_clf.predict(X_test)

# Bondad del ajuste
print(accuracy_score(y_test, y_pred))

0.98


In [41]:
# Se obtiene un valor de accuracy muy alto

In [43]:
# Apartado b)
# Se repite el apartado anterior, pero con bootstrap=False como argumento en la creación del objeto BaggingClassifier para que el método de ensamble
# sea de tipo pasting.

# Se crea objeto BaggingClassifier
# Con bootstrap = False se establece que el método de ensamble es tipo pasting
pas_clf = BaggingClassifier(DecisionTreeClassifier(random_state=3), n_estimators=500, max_samples=75, bootstrap=False, random_state=3)

# Se ajusta el modelo
pas_clf.fit(X_train, y_train)

# Estimaciones del modelo sobre el conjunto de validación
y_pred_pas = pas_clf.predict(X_test)

# Bondad del ajuste
print(accuracy_score(y_test, y_pred_pas))

0.98


In [45]:
# La medida del accuracy es la misma tanto con el método de bagging como con el de pasting.

In [47]:
# Apartado c)
from sklearn.ensemble import RandomForestClassifier # Para crear Random Forest

# Se crea objeto Random Forest
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=4, random_state=3, max_samples=75)

# Se ajusta el modelo
rnd_clf.fit(X_train, y_train)

# Estimaciones del modelo sobre el conjunto de validación
y_pred_rf = rnd_clf.predict(X_test)

# Bondad del ajuste
print(accuracy_score(y_test, y_pred_rf))

0.98


In [50]:
# Se obtiene el mismo valor para accuracy que en los dos apartados anteriores, por lo que no se puede diferenciar un método de ensamble como el mejor.
# Comparamos los resultados obtenidos con los métodos de ensamble con aquellos que se obtendrían con un modelo simple de árbol de decisión.
tree_clf = DecisionTreeClassifier(random_state=3, max_depth=3)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

0.9


In [ ]:
# Se obtiene un menor valor para accuracy con el modelo de árbol de decisión, lo cual pone de manifiesto que es un peor modelo que los anteriores.